In [ ]:
# Link to Original Notebook

# https://colab.research.google.com/drive/1xeGth-p18kkae4n0iT-CqzaZuiFEMKpb?authuser=0#scrollTo=T6JSIvjOa_B8

In [ ]:
# Upload Shade Pictures

from google.colab import files
import os

# Create input folder
input_folder = "/content/input_pictures"
os.makedirs(input_folder, exist_ok=True)

# Upload prompt — this waits for YOU to upload:
print("📂 Please upload your pictures now!")
uploaded = files.upload()

# Move uploaded files to input folder
for filename in uploaded.keys():
    os.rename(filename, os.path.join(input_folder, filename))
    print(f"✅ Saved: {filename} → {input_folder}")

print("\n✅ All files uploaded! Ready to process.")


In [ ]:
# Crop Each Screenshot and Rename Them

import os
from PIL import Image

# Input/output folders
input_folder = "/content/input_pictures"
output_folder = "/content/output_cropped"
os.makedirs(output_folder, exist_ok=True)

# Crop region for MAIN picture
main_crop_box = (600, 100, 1400, 880)  # Example — you can adjust!

# Month names
month_names = {
    1: "Jan",
    2: "Feb",
    3: "Mar",
    4: "Apr",
    5: "May",
    6: "Jun",
    7: "Jul",
    8: "Aug",
    9: "Sep",
    10: "Oct",
    11: "Nov",
    12: "Dec"
}

# Time labels per iteration (6 steps of 2 hours):
time_labels = [
    (8, "08"),
    (10, "10"),
    (12, "12"),
    (14, "14"),
    (16, "16"),
    (18, "18")
]

# Sort filenames so the order matches the batch!
all_files = sorted(os.listdir(input_folder))
file_index = 0

# Main loop:
for month in range(1, 13):
    month_name = month_names[month]
    day = 15

    for hour, time_label in time_labels:
        if file_index >= len(all_files):
            print("⚠️ Not enough files uploaded!")
            break

        filename = all_files[file_index]
        input_path = os.path.join(input_folder, filename)
        print(f"\nProcessing: {filename} → Month {month_name}, Day {day}, Time {time_label}")

        # Open image
        img = Image.open(input_path)

        # Crop main image
        main_cropped = img.crop(main_crop_box)

        # Build output filename:
        new_filename = f"calendar_logo_{month_name}_{day}_{time_label}.png"
        output_path = os.path.join(output_folder, new_filename)

        # Save
        main_cropped.save(output_path)
        print(f"✅ Saved: {output_path}")

        # Next file
        file_index += 1

print("\n🎉 Batch done!")


In [ ]:
# Make Cropped Images into Shade PNGs and Shade TXTs

import os
import re
import cv2
import numpy as np
from matplotlib import pyplot as plt
from google.colab import files

# Set input and output folders
input_folder = '/content/output_cropped'
output_folder = '/shade_txt'

# Create output folder
os.makedirs(output_folder, exist_ok=True)

# Month mapping
month_map = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05',
             'Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10',
             'Nov':'11', 'Dec':'12'}

# Helper: Convert filename to MMHH
def filename_to_MMHH(filename):
    # Expected: 'calendar_logo_Jan_15_08'
    match = re.match(r'calendar_logo_([A-Za-z]+)_(\d{1,2})_(\d{2})', filename)
    if not match:
        print(f"⚠️ Filename format not recognized: {filename}")
        return None

    month_str, day_str, hour_str = match.groups()
    MM = month_map.get(month_str, '00')
    HH = hour_str
    return f"{MM}{HH}"

# Threshold value (adjust if needed)
threshold_value = 127

# Process images
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
        image_path = os.path.join(input_folder, filename)

        # Load grayscale
        img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Threshold to binary 1/0
        _, binary_img = cv2.threshold(img_gray, threshold_value, 1, cv2.THRESH_BINARY_INV)

        # Convert filename
        base_filename = os.path.splitext(filename)[0]
        MMHH = filename_to_MMHH(base_filename)

        if MMHH:
            txt_filename = f"{MMHH}.txt"
            txt_path = os.path.join(output_folder, txt_filename)

            # Save as txt
            np.savetxt(txt_path, binary_img, fmt='%d')

            # Print 1/0 map
            print(f"=== {txt_filename} ===")
            for row in binary_img:
                print(' '.join(map(str, row.tolist())))
            print("\n")

print("✅ All images processed. TXT files saved in 'shade_txt' folder.")

# Zip the output folder for download
!zip -r /shade_txt.zip /shade_txt

# Provide download link
files.download('/shade_txt.zip')


In [ ]:
# Upscale Shade TXTs to Match Walkability Map Resolution

# STEP 1 - Upload ZIP, extract, upscale all txt files, save to folder

import os
import zipfile
import numpy as np
from google.colab import files
from tqdm import tqdm

# 1️⃣ Upload ZIP file
uploaded = files.upload()

# 2️⃣ Unzip to folder
zip_filename = list(uploaded.keys())[0]
extract_folder = 'shade_txts'
os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

print(f'Extracted to: {extract_folder}')

# 3️⃣ Prepare output folder
output_folder = 'upscaled_shade'
os.makedirs(output_folder, exist_ok=True)

# 4️⃣ Set scaling factor
scale_x = 10
scale_y = 10

# 5️⃣ Process each txt file
txt_files = [f for f in os.listdir(extract_folder) if f.endswith('.txt')]
txt_files.sort()  # optional: sort by name

print(f'Found {len(txt_files)} txt files.')

for txt_file in tqdm(txt_files):
    # Load original shade map
    shade = np.loadtxt(os.path.join(extract_folder, txt_file), dtype=int)

    # Upscale
    shade_upscaled = np.repeat(np.repeat(shade, scale_y, axis=0), scale_x, axis=1)

    # Save to output folder with same name
    output_path = os.path.join(output_folder, txt_file)
    np.savetxt(output_path, shade_upscaled, fmt='%d')

print('Upscaling complete! All files saved to:', output_folder)

# STEP 2 - Zip the folder and download

import shutil

shutil.make_archive('upscaled_shade', 'zip', 'upscaled_shade')

# Offer download
files.download('upscaled_shade.zip')


In [ ]:
# Process Walkability Map

import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab import files
from PIL import Image

# --- Function to export TXT ---
def save_walkability_as_txt(walkability_map, filename="walkability_map.txt"):
    np.savetxt(filename, walkability_map, fmt='%d', delimiter=' ')
    print(f"\n✅ Walkability map saved as plain text: {filename}")
    files.download(filename)

# --- Function to export JPG ---
def save_walkability_as_jpg(walkability_map, filename="walkability_map.jpg"):
    img_255 = (walkability_map * 255).astype(np.uint8)
    img_pil = Image.fromarray(img_255)
    img_pil.save(filename)
    print(f"\n✅ Walkability map saved as JPG: {filename}")
    files.download(filename)

# --- Upload Image ---
uploaded = files.upload()
filename = next(iter(uploaded.keys()))

# --- Read Image ---
img_bgr = cv2.imread(filename)
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

# --- Prepare ---
height, width, _ = img_bgr.shape
walkability_map = np.zeros((height, width), dtype=np.uint8)

b = img_bgr[:, :, 0].astype(int)
g = img_bgr[:, :, 1].astype(int)
r = img_bgr[:, :, 2].astype(int)

# --- RULE: White = Walkable ---
# Allow a tolerance because "perfect white" is rare:
# Example: R,G,B > 240 → very white
white_mask = (r > 240) & (g > 240) & (b > 240)

# --- Apply ---
walkability_map[white_mask] = 1  # walkable
# All others stay 0

# --- Display ---
plt.figure(figsize=(12, 6))

# Original
plt.subplot(1, 2, 1)
plt.imshow(img_rgb)
plt.title("Original Map")
plt.axis('off')

# Walkability Map
plt.subplot(1, 2, 2)
plt.imshow(walkability_map, cmap='gray', vmin=0, vmax=1)
plt.title("Walkability Map (1=walkable, 0=unwalkable)")
plt.axis('off')

plt.show()

# --- Show Array ---
print("\n=== Walkability 2D Array ===")
print(walkability_map)

# --- Save and Download ---
save_walkability_as_txt(walkability_map, filename="walkability_map.txt")
save_walkability_as_jpg(walkability_map, filename="walkability_map.jpg")
